In [ ]:
# import libraries
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [ ]:
# Assign sqlite database to db variable
db = sqlite3.connect('../data/hop_teaming.sqlite')

In [ ]:
# Check the tables in the database
tables = db.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in tables:
    print(table[0])

In [ ]:
col = db.execute("PRAGMA table_info(referrals)")
print(col.fetchall())

In [ ]:
query = """
WITH nashville_provider AS (
    SELECT n.*

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
)

SELECT COUNT(*) AS tot_rows
    , COUNT(DISTINCT n.npi) AS unique_npis
    , COUNT(CASE WHEN s.npi IS NOT NULL THEN s.npi ELSE NULL END) AS npis_with_specialty
    , COUNT(CASE WHEN rto.to_npi IS NOT NULL THEN rto.to_npi ELSE NULL END) AS npis_who_refer_out
    , COUNT(CASE WHEN rfrom.to_npi IS NOT NULL THEN rfrom.to_npi ELSE NULL END) AS npis_who_get_referrals
    
FROM nashville_providers AS n

LEFT JOIN specialty AS s
    ON s.npi = n.npi
    
LEFT JOIN nashville_referrals AS rto
    ON rto.to_npi = n.npi
    
LEFT JOIN nashville_referrals AS rfrom
    ON rfrom.from_npi = n.npi
"""

In [ ]:
nashville_providers = pd.read_sql(query, db)

In [ ]:
nashville_providers.head()